<a href="https://colab.research.google.com/github/BowieSteutel/acc-nlp-firecodes/blob/main/2_Building_Information_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Module 2 - Building Information Processing**



*This module uses the ontologies to convert the IFC use cases to RDF.*

# **Prepare libraries**

In [14]:
# Import standard libraries
from collections import defaultdict # for empty dictionary entries
from datetime import datetime # for determining the current time for the base URI

In [15]:
# IfcOpenShell (for reading IFC files)
!pip install ifcopenshell pandas requests owlrl --quiet
import ifcopenshell as ios
import ifcopenshell.guid
import ifcopenshell.util
import ifcopenshell.util.element
import ifcopenshell.util.unit
import ifcopenshell.util.pset

In [16]:
# RDFLib (for RDF conversion)
!pip install rdflib --quiet
import rdflib
from rdflib import Graph, Namespace, Literal, URIRef

In [17]:
# pySHACL (for materialization)
!pip install pyshacl --quiet
import pyshacl

---
# **Prepare inputs & outputs**

In [18]:
# @title Change root directory (update after downloading)

root_directory = "/content/drive/MyDrive/FINAL_CODE_THESIS" #  @param {"type":"string", "placeholder":""}
import sys
from pathlib import Path
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    %cd {root_directory}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/FINAL_CODE_THESIS


In [19]:
# @title Define filepaths
# Use case models to convert & output name
input_correct = "input/use_case_correct.ifc" # @param {type:"string", placeholder:"(ifc)"}
output_correct = "output/use_case_correct.ttl"  # @param {type:"string", placeholder:"(ttl)"}
input_incorrect = "input/use_case_incorrect.ifc" # @param {type:"string", placeholder:"(ifc)"}
output_incorrect = "output/use_case_incorrect.ttl" # @param {type:"string", placeholder:"(ttl)"}

# Ontologies (for materialization)
ont_path_isolated = "input/custom_ontology_isolated.ttl" # @param {type:"string"}
ont_path_alignment = "input/ontology_alignment.ttl" # @param {type:"string"}

## Define namespaces

In [20]:
# load default namespaces
from rdflib.namespace import RDF, RDFS, OWL, XSD

# define other namespaces
#DCE = "https://purl.org/dc/elements/1.1/"
#VANN = "https://purl.org/vocab/vann/"
#CC = "https://creativecommons.org/ns#"
BOT = "https://w3id.org/bot#"
BEO = "https://w3id.org/beo#"
#BEO = "https://pi.pauwel.be/voc/buildingelement#"
#MEP = "https://pi.pauwel.be/voc/distributionelement#"
#GEOM = "https://w3id.org/geom#"
PROPS = "https://w3id.org/props#"
#QUDT = "http://qudt.org/schema/qudt/"
QUDT = "http://qudt.org/schema/shacl/qudt/"
UNIT = "https://qudt.org/vocab/unit/"
PSET = "https://example.org/pset#"
MAT = "https://example.org/material#"
#IFC = "https://standards.buildingsmart.org/IFC/DEV/IFC4_1/FINAL/"
# IFC = "https://w3id.org/ifc/IFC4#"
IFC = "https://w3id.org/ifc/IFC4X3_ADD2#"
EX = "https://example.org/ns#"
#EMPTY = "https://example.org/"
#SCHEMA = "https://schema.org/"

---

# **Prepare file conversion functions**

### Headers

In [21]:
def RDF_header(title):
  return f'''

#################################################################
#\t{title}
#################################################################

'''

### cleanString

In [22]:
# properties
def cleanString(name):
    name = ''.join(x for x in name.title() if not x.isspace())
    name = name.replace('\\', '')
    name = name.replace('/', '')
    name = name.replace('(', '_')
    name = name.replace(')', '')
    # NEW DEFINITIONS FOR RDF SUPPORT:
    name = name.replace(':', '_')
    name = name.replace(';', '_')
    name = name.replace('[', '_')
    name = name.replace(']', '_')
    name = name.replace('.', '_')
    name = name.replace(',', '_')
    return name

In [23]:
# Load unit vocabulary from QUDT
unit_vocab = Graph()
# unit_vocab.parse("https://qudt.org/2.1/vocab/unit.ttl", format="turtle")  # This URL does not use qudt:scalingOf
unit_vocab.parse("https://qudt.org/vocab/unit/", format="turtle")  # This URL uses qudt:scalingOf

<Graph identifier=N6e9802bdeae04968970c99afe24513cf (<class 'rdflib.graph.Graph'>)>

### convert2QUDT

In [24]:
# Find labels in qudt

def convert2QUDT(original_unit, ifc_unit, ifc_name):
  ifc_unit = str(ifc_unit).lower() # make case-insensitive
  ifc_unit = ifc_unit.replace("pound", "pound( MASS)?") #make sure pounds are matched
  #ifc_unit = ifc_unit.replace("meter", "metre") #make sure meters (en-US) are matched as metres (en)
  ifc_unit = ifc_unit.replace("_", " ") #make sure meters (en-US) are matched as metres (en)

  query = """
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX unit: <""" + UNIT + """>

  SELECT ?unitname ?factor
  WHERE {
    ?unitname rdfs:label ?label .
    ?unitname qudt:conversionMultiplier ?factor .
      FILTER regex(?label, "^""" + ifc_unit + """$"@en, "i" )
  }"""

  #find matches
  results = unit_vocab.query(query)

  if not results:
    return None


  for row in results:
    if not row.unitname:
      continue  # Skip if unit is None
    # Assign a value to qudt_unit before using it
    qudt_unit = str(row.unitname)

    #if row.unitname.startswith("<"+str(UNIT)):  # Check if it's a QUDT unit
    qudt_unit = "unit:"+qudt_unit.split('/')[-1]  # Return prefix format
    #else:
      #qudt_unit = f"<{row.unitname}>" # return as URI
    # if (row.factor, 3) == 1:
    #   factor =  int(1)
    # else:
    #   factor = row.factor
    return qudt_unit#, factor
print(convert2QUDT(None, None, "Luminous Efficacy"))
print(convert2QUDT("dB", None, "SOUNDPOWER"))
print(convert2QUDT(None, "Kilogram", None))

None
None
unit:KiloGM


### findUnits

In [25]:
dict_exponents = {
    "None": "",
    None : "",
    1 : "",
    2 : "Square(?:d)?",
    3 : "Cub(?:ic|ed)",
    4 : "Quartic",
    5 : "Quintic",
    6 : "Sextic",
}


def cleanUnitName(unit):
    unit_name = unit.replace("_", " ")
    unit_name = unit_name.title()
    unit_name = unit_name.replace("Meter", "Metre")
    #unit_name = unit_name.replace(" Per ", " per ")
    #unit_name = unit_name.replace(" Per ", "(s?) per ")
    unit_name += "(s?)"

    return unit_name

def findUnitName(unit):
    name = str(unit.Name)
    try:
      prefix = unit.Prefix
      if prefix != "None":
          return cleanUnitName(prefix+name)
      else:
          return cleanUnitName(name)
    except:
        return cleanUnitName(name)


def combineUnits(d_u):
    #comb = ""
    #print(len(d_u))
    if len(d_u) == 0:
        return ""
    elif len(d_u) == 1:
        return d_u[0]
    elif len(d_u) == 2:
        return f"({d_u[0]} {d_u[1]}|{d_u[1]} {d_u[0]})"
    elif len(d_u) == 3:
        return f"({d_u[0]} {d_u[1]} {d_u[2]}|{d_u[0]} {d_u[2]} {d_u[1]}|{d_u[1]} {d_u[0]} {d_u[2]}|{d_u[1]} {d_u[2]} {d_u[0]}|{d_u[2]} {d_u[0]} {d_u[1]}|{d_u[2]} {d_u[1]} {d_u[0]})"
    else:
        return "ERROR"
        #comb = " ".join([x for x in d_u])

def standardizeMeasurements(unit, name, factor, from_unit, to_unit):
    from_unit = ""
    to_unit = ""
    match unit.is_a():
        case "IfcSIUnit":
            from_unit_name = str(unit.Name)
            try:
                factor *= ios.util.unit.prefixes[unit.Prefix]
                from_unit += cleanUnitName(str(unit.Prefix) + from_unit_name)
            except:
                from_unit += cleanUnitName(from_unit_name)
            if unit.Name == "GRAM":
                factor *= 0.001 #kilogram is only SI unit with prefix in the base unit
                to_unit += cleanUnitName("Kilo" + from_unit_name)
            else:
                to_unit += cleanUnitName(from_unit_name)
        case "IfcConversionBasedUnit":
            factor *= unit.ConversionFactor.ValueComponent[0]
            from_unit += findUnitName(unit)
            to_unit += findUnitName(unit.ConversionFactor.UnitComponent)

        case "IfcDerivedUnit":
            if unit.UserDefinedType:
                name = unit.UserDefinedType
                #print(name)
            derived_num = []
            derived_denom = []
            derived_original = ""
            for element in unit.Elements: #decompose derived units
                element_name, element_factor, element_from_unit, element_to_unit = standardizeMeasurements(element, element.Unit, factor, "", "")
                derived_original += element_from_unit + " "
                factor *= element_factor**element.Exponent

                # do the following:
                # if factor - something: [unit 1](s)? per [unit2]
                # if factor bigger than 1: square, cubic, quartic, quintic, sextic, heptic, octic
                # meter = metre
                # if multiple multiplied next to each other: both order should be tried, e.g.: N/Ks² and N/s²K
                # also try exponent on both sides, e.g. square metre & metre squared, cubic metre & metre cubed
                # no numerator? leave out
                # no denominator? replace "per" with "(?:Reciprocal |per )?"

                derived_exp = str(dict_exponents.get(abs(element.Exponent)))

                derived_name = str(findUnitName(element_name))
                if abs(element.Exponent) == 2:
                    derived_element = f"({derived_exp} {derived_name}|{derived_name} {derived_exp})"
                elif derived_exp != "":
                    derived_element = f"({derived_exp} {derived_name})"
                else:
                    derived_element = derived_name

                # add elements to numerator or denominator based on exponent
                if abs(element.Exponent) == element.Exponent:
                    derived_num.append(derived_element)
                else:
                    derived_denom.append(derived_element)
            from_unit = derived_original
            derived_num_comb = combineUnits(derived_num)
            derived_denom_comb = combineUnits(derived_denom)
            if not derived_num_comb and not derived_denom_comb:
                to_unit = from_unit
            if not derived_num_comb:
                to_unit = "(?:Reciprocal |per )?"+derived_denom_comb
            elif not derived_denom_comb:
                to_unit = derived_num_comb
            elif derived_num_comb and derived_denom_comb:
                to_unit = f"(?:{derived_num_comb} per {derived_denom_comb})"
            return name, factor, from_unit, to_unit

    return name, factor, from_unit, to_unit

def findUnits(model):
    units = model.by_type("IfcUnitAssignment")[0]
    dict_units = dict()
    for u in units.Units:
        name, factor, from_unit, to_unit = standardizeMeasurements(u, u.UnitType[:-4], 1, "", "")
        dict_units[name] = [factor, convert2QUDT(from_unit, to_unit, name)]
        #print(name, factor, "|", from_unit, "-->", to_unit)
    return dict_units


#also check if user defined type, in that case just take that name. Do the same thing later with Ifc..Measure vs

### printProperties

In [26]:
def printValue(name, value, output, tab, unit):
    output += tab*"\t"+"props:"+name
    if isinstance(value, bool):
        output += " \""+str(value) +"\"^^xsd:boolean "
    elif isinstance(value, int):
        # If measurement has a unit, make a blank property node. Else, skip this step
        if unit:
            output += " [\n"+(tab+1)*"\t"+"qudt:NumericValue"
        output += " \""+str(value) +"\"^^xsd:int "
        if unit:
            output += ";\n"+(tab+1)*"\t"+"qudt:hasUnit "+str(unit)+" ;\n"+tab*"\t"+"] " #" ;"
            #output += "\n"+(tab+1)*"\t"+"] ; "
    elif isinstance(value, float):
        # If measurement has a unit, make a blank property node. Else, skip this step
        if unit:
            output += " [\n"+(tab+1)*"\t"+"qudt:NumericValue"
        output += " \""+str(value) +"\"^^xsd:double "
        if unit:
            output += ";\n"+(tab+1)*"\t"+"qudt:hasUnit "+str(unit)+" ;\n"+tab*"\t"+"] " #" ;"
            #output += "\n"+tab*"\t"+"] ; "
    else:
        output += " \""+str(value) +"\"^^xsd:string "
    return output

def printProperties(pset_name, properties, output, tab, model_units):
    for prop_name, prop in properties.items():
        if prop_name == "id":
            continue

        standard_unit = None

        if type(prop) == dict:
            if pset_name == "BaseQuantities": #QTO
                prop_value = prop.get('value')
                prop_unit = prop.get('class')
                if prop_unit[:11] == "IfcQuantity":
                    prop_standardized = model_units.get(prop_unit[11:].upper())
                    prop_value *= prop_standardized[0]
                    standard_unit = prop_standardized[1]
            else: #PSET
                prop_value = prop.get('value')
                prop_unit = prop.get('value_type')
                if prop_unit[-12:] == "RatioMeasure": #special case for ratios, which have no unit
                    prop_value = prop_value
                    standard_unit = "unit:UNITLESS"
                elif prop_unit[-7:] == "Measure": #standardize measures
                    try:
                        prop_standardized = model_units.get(prop_unit[3:-7].upper())
                        prop_value *= prop_standardized[0]
                        standard_unit = prop_standardized[1]
                    except:
                        print("ERROR CONVERTING", prop_unit, prop_value)

        else:
            prop_value = prop

        prop_name = cleanString(prop_name)
        output += ";\n"
        #if qudt_unit:
        output = printValue(prop_name, prop_value, output, tab+1, standard_unit)
        #else:
            #output = printValue(prop_name, prop_value, output, tab+1, None)
    return output

### writePropertySets

In [27]:
def writePropertySets(instance_name, psets, output, model_units):
    for name, properties in psets.items():
        pset_name = cleanString(name)

        # remove * from Pset_*Common for generalization purposes (original name still stored with rdfs:label)
        if pset_name.lower().endswith("common"):
            pset_name = "Common"
        output += ";\n"
        output += f"\tpset:{pset_name} [\n"
        output += f"\t\trdfs:label \"{name}\"^^xsd:string "# ;\n"

        # for prop in properties:
        #   output = printProperties(name, prop, output, 1)
        output = printProperties(name, properties, output, 1, model_units)
        output += ";\n\t] "
        #output += "\n\t\t] "
        # if output[-2:] == "] ":
        #   ...
        # else:
        #   #output += "] "
    output += ". \n\n"
    return(output)


### loadBEO & IFC2BEO

In [28]:
# prepare BEO
def loadBEO():
    # Load BEO ontology
    beo_file = "https://cramonell.github.io/beo/actual/ontology.ttl"

    g_beo = Graph()
    g_beo.parse(beo_file, format="turtle")

    # Extract all BEO classes
    beo_classes = {
        str(cls).split("#")[-1]: cls for cls in g_beo.subjects(predicate=None, object=URIRef("http://www.w3.org/2002/07/owl#Class"))
    }

    return beo_classes

def IFC2BEO(ifc_name, beo_classes):
    name = ifc_name.split('Ifc')[-1] # Remove "(ifc:)Ifc"
    if name in beo_classes:
        return "beo:"+beo_classes[name].split('#')[1]
    else:
        if ":" in ifc_name: # if name already has a format, return it as is
            return ifc_name
        else:           # otherwise, return ifc:ifc_name
            return "ifc:"+ifc_name

### findCompartments

In [29]:
# finding ids and types of compartments in model
def findCompartments(model):
    compartments = dict()
    # find the name and code of each compartment
    for c in model.by_type("IfcSpace"):
        if c.Name:
            if "fire" in c.LongName.lower():
                #l_compartments.append([c.Name, None, c.LongName, c.id()])
                if "protected sub" in c.LongName.lower():
                    compartments[c.Name] = ['ex:ProtectedSubFireCompartment', c.id()]
                elif "sub" in c.LongName.lower():
                    compartments[c.Name] = ['ex:SubFireCompartment', c.id()]
                else:
                    compartments[c.Name] = ['ex:FireCompartment', c.id()]


    #sort compartments by name
    compartments = dict(sorted(compartments.items()))
    #l_compartments = sorted(l_compartments, key=lambda x: x[0])

    return compartments

### writeSites

In [30]:
def writeSites(model, model_units):
  output = ""
  for s in model.by_type("IfcSite"):
    instance_name = f"inst:Site_{str(s.id())}"
    output += instance_name + "\n"
    output += "\ta bot:Site ;" + "\n"
    if(s.Name):
        output += "\trdfs:label \""+s.Name+"\"^^xsd:string ;" + "\n"
    if(s.Description):
        output += "\trdfs:comment \""+s.Description+"\"^^xsd:string ;" + "\n"
    # output += "\tprops:hasGuid \""+ ios.guid.expand(s.GlobalId) +"\"^^xsd:string ;" + "\n"
    output += "\tprops:hasCompressedGuid \""+ s.GlobalId +"\"^^xsd:string "
    for reldec in s.IsDecomposedBy:
        if reldec is not None:
            for b in reldec.RelatedObjects:
                output += ";\n"
                output += "\tbot:hasBuilding inst:Building_"+ str(b.id()) + " "


    #psets = ios.util.element.get_psets(s)
    psets = ios.util.element.get_psets(s, verbose=True)
    output = writePropertySets(instance_name, psets, output, model_units)

  return output

### writeBuildings

In [31]:
def writeBuildings(model, model_units, model_compartments):
    output = ""
    for b in model.by_type("IfcBuilding"):
        instance_name = f"inst:Building_{str(b.id())}"
        output += instance_name + "\n"
        output += "\ta bot:Building ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"
        # output += "\tprops:hasGuid \""+ ios.guid.expand(b.GlobalId) +"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "
        for reldec in b.IsDecomposedBy:
            if reldec is not None:
                for st in reldec.RelatedObjects:
                    output += ";\n"
                    output += "\tbot:hasStorey inst:Storey_"+ str(st.id()) + " "
        #psets = ios.util.element.get_psets(b)
        psets = ios.util.element.get_psets(b, verbose=True)
        output = writePropertySets(instance_name, psets, output, model_units)
    return output

### writeStoreys

In [32]:
def writeStoreys(model, model_units, model_compartments):
    output = ""
    for b in model.by_type("IfcBuildingStorey"):
        instance_name = f"inst:Storey_{str(b.id())}"
        output += instance_name + "\n"
        output += "\ta bot:Storey ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "
        for reldec in b.IsDecomposedBy:
            if reldec is not None:
                for sp in reldec.RelatedObjects:
                    # make distinction between space and compartment
                    sp_id = sp.id()
                    if sp_id in [x[1] for x in model_compartments.values()]:
                        None
                    if sp_id not in [x[1] for x in model_compartments.values()]:
                        output += ";\n"
                        output += "\tbot:hasSpace inst:Space_"+ str(sp.id()) + " "
        for relcontains in b.ContainsElements:
            if relcontains is not None:
                for el in relcontains.RelatedElements:
                    el_type = el.is_a()
                    output += ";\n"
                    output += "\tbot:containsElement inst:" + el_type.replace("Ifc", "") + "_" + str(el.id()) + " "

        psets = ios.util.element.get_psets(b, verbose=True)
        output = writePropertySets(instance_name, psets, output, model_units)
    return output

### writeCompartments

In [33]:
def writeCompartments(model, model_units, model_compartments):
    output = ""
    # go through compartment list in order
    for c_id in [x[1] for x in model_compartments.values()]:
        c = model[c_id]
        instance_name = f"inst:Compartment_{str(c_id)}"
        output += instance_name + "\n"
        output += f"\ta {str(model_compartments[c.Name][0])} ;\n"
        if(c.Name):
            output += "\trdfs:label \""+c.Name+"\"^^xsd:string ;" + "\n"
        if(c.Description):
            output += "\trdfs:comment \""+c.Description+"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ c.GlobalId +"\"^^xsd:string "
        try:
            parent_code = str('.'.join(str(c.Name).split('.')[:-1]))
            if parent_code != c.Name:
                output += f';\n\tex:partOfCompartment inst:Compartment_{str(model_compartments[parent_code][1])} '
        except:
            None

        psets = ios.util.element.get_psets(c, verbose=True)
        output = writePropertySets(instance_name, psets, output, model_units)
    return output

### writeSpaces

In [34]:
def writeSpaces(model, model_units, model_compartments):
    output = ""
    for s in model.by_type("IfcSpace"):
        if s.id() not in [x[1] for x in model_compartments.values()]: #filter out compartments
            instance_name = f"inst:Space_{str(s.id())}"
            output += instance_name + "\n"
            output += "\ta bot:Space ;\n"
            if(s.Name):
                output += "\trdfs:label \""+s.Name+"\"^^xsd:string ;" + "\n"
            if(s.Description):
                output += "\trdfs:comment \""+s.Description+"\"^^xsd:string ;" + "\n"
            output += "\tprops:hasCompressedGuid \""+ s.GlobalId +"\"^^xsd:string "
            try:
                space_in_c = ios.util.element.get_psets(s, verbose=True)['Other']['LocatedInCompartment']['value']
                output += f';\n\tex:locatedInCompartment inst:Compartment_{str(model_compartments[space_in_c][1])} '
            except:
                None
            for relbounded in s.BoundedBy:
                if relbounded is not None:
                    el = relbounded.RelatedBuildingElement
                    if el is not None:
                        el_type = el.is_a()
                        output += ";\n"
                        output += "\tbot:adjacentElement inst:" + el_type.replace("Ifc", "") + "_" + str(el.id()) + " "
            for relcontains in s.ContainsElements:
                if relcontains is not None:
                    for el in relcontains.RelatedElements:
                        output += ";\n"
                        output += "\tbot:containsElement inst:" + el_type.replace("Ifc", "") + "_" + str(el.id()) + " "

            psets = ios.util.element.get_psets(s, verbose=True)
            output = writePropertySets(instance_name, psets, output, model_units)
    return output

### writeElements

In [35]:
def get_materials(element, model):
    materials = set()

    # Direct material assignment
    material_relations = model.get_inverse(element)
    for rel in material_relations:
        if "IfcRelAssociatesMaterial" in rel.is_a():
            material = rel.RelatingMaterial
            if material:
                materials.add(material)
    if materials:
        return list(materials)[0] # flatten set

def writeElements(model, model_units, beo_classes):
    output = ""
    for b in model.by_type("IfcElement"):
        element_type = b.is_a()
        element_type_clean = element_type.replace("Ifc", "")  # Remove "Ifc" prefix for better readability
        instance_name = f"inst:{element_type_clean}_{b.id()}"
        output += instance_name + "\n"
        output += f"\ta bot:Element , {IFC2BEO(element_type, beo_classes)} ;" + "\n"

        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "

        material = get_materials(b, model)
        if material:
            # If it's a simple IfcMaterial
            if material.is_a() == "IfcMaterial":
                output += f";\n\tex:hasMaterial inst:Material_{material.id()} "

            # If it's a layered material set
            elif material.is_a() == "IfcMaterialLayerSetUsage":
                layer_set = material.ForLayerSet
                if layer_set and hasattr(layer_set, "MaterialLayers"):
                    for layer in layer_set.MaterialLayers:
                        if layer.Material:
                            output += f";\n\tex:hasMaterial inst:Material_{layer.Material.id()} "

            # If it's a material constituent set
            elif material.is_a() == "IfcMaterialConstituentSet":
                if hasattr(material, "MaterialConstituents"):
                    for constituent in material.MaterialConstituents:
                        if constituent.Material:
                            output += f";\n\tex:hasMaterial inst:Material_{constituent.Material.id()} "

        for relvoids in b.HasOpenings:
            if relvoids is not None:
                el = relvoids.RelatedOpeningElement
                for relfills in el.HasFillings:
                    if relfills is not None:
                        filler = relfills.RelatedBuildingElement
                        el_type = el.is_a()
                        output += ";\n"
                        output += "\tbot:hasSubElement inst:" + el_type.replace("Ifc", "") + "_" + str(el.id()) + " "

        for relvoids in b.HasOpenings:
            if relvoids is not None:
                el = relvoids.RelatedOpeningElement
                for relfills in el.HasFillings:
                    if relfills is not None:
                        filler = relfills.RelatedBuildingElement
                        el_type = el.is_a()
                        output += ";\n"
                        output += "\tbot:hasSubElement inst:" + el_type.replace("Ifc", "") + "_" + str(el.id()) + " "

        psets = ios.util.element.get_psets(b, verbose=True)
        output = writePropertySets(instance_name, psets, output, model_units)
    return output

### writeMaterials

In [36]:
def writeMaterials(model, model_units):
    output = ""
    for m in model.by_type("IfcMaterial")[:]:
        instance_name = f"inst:Material_{m.id()}"
        output += instance_name + "\n"
        output += f"\ta ex:Material ;" + "\n"

        if(m.Name):
            output += "\trdfs:label \""+m.Name+"\"^^xsd:string ;" + "\n"
        if(m.Description):
            output += "\trdfs:comment \""+m.Description+"\"^^xsd:string ;" + "\n"
        output = output[:-2] # remove last ; (needs to be done because of missing GuID and previous coding conventions)
        psets = ios.util.element.get_psets(m, verbose=True)

        # then, write them to RDF
        output = writePropertySets(instance_name, psets, output, model_units)
    return output

### writeAdjacencies

In [37]:
def writeAdjacencies(model, model_compartments):
    sp_el_adj = defaultdict(set) # Dictionary to map elements to spaces
    # Group spaces by shared boundary elements
    for boundary in model.by_type("IfcRelSpaceBoundary"):
        space = boundary.RelatingSpace
        element = boundary.RelatedBuildingElement

        if space and element and element.is_a() not in ['IfcSlab', 'IfcRoof']: #only look for horizontal adjacency
            element_name = element.is_a().replace('ifc', '')
            sp_el_adj[f"inst:{element_name}_{element.id()}"].add(space.id())

    # Use shared elements to determine adjacency
    sp_sp_adj = defaultdict(list) # Dictionary to store adjacency between spaces
    for element_id, space_ids in sp_el_adj.items():
        space_ids = list(space_ids)
        for i in range(len(space_ids)):
            for j in range(i + 1, len(space_ids)):
                sp_sp_adj[space_ids[i]].append(space_ids[j])
                sp_sp_adj[space_ids[j]].append(space_ids[i])

    # Remove duplicates from list
    sp_sp_adj = [(id, list(set(data))) for (id, data) in sp_sp_adj.items()]

    # Write compartment-compartment adjacencies
    # The difference in the space-compartment containment indicates the adjacency of compartments
    # (since ex:adjacentCompartment is disjoint with ex:locatedInCompartment)

    # find out which which compartment each space is in, including parent compartments
    space_in_c = dict()
    for s in model.by_type("IfcSpace"):
        if s.id() not in [x[1] for x in model_compartments.values()]:
            # find the compartment containment property
            space_id = s.id()
            psets = ios.util.element.get_psets(s, verbose=True)
            # Check if 'Other' and 'LocatedInCompartment' keys exist
            if 'Other' in psets and 'LocatedInCompartment' in psets['Other']:
                c_code = psets['Other']['LocatedInCompartment']['value']
                c_id = model_compartments[c_code][1]

                # save to dictionary
                space_in_c[space_id] = [c_id]

                # find parent compartments and add to dictionary
                c_parent = c_code
                while '.' in c_parent:
                    c_parent = '.'.join(str(c_parent).split('.')[:-1])
                    space_in_c[space_id].append(model_compartments[c_parent][1])
            else:
                # Handle the case where the keys are missing
                print(f"Warning: Space {s.id()} is missing 'LocatedInCompartment' property or it's not in property set 'Other'")

    # find out which spaces are in each compartment
    spaces_per_c = defaultdict(list)
    for c_id in [x[1] for x in model_compartments.values()]:
        for s in space_in_c.items():
            if c_id in s[1]:
                spaces_per_c[c_id].append(s[0])

    adj_spaces = defaultdict(list)
    adj_compartments = defaultdict(list)
    con_compartments = defaultdict(list)

    # Function for flattening a nested list into a single list.
    def flatten(l):
      return [item for sublist in l for item in sublist]
    # find adjacent spaces for each contained space and find their respective compartment(s)
    for c in spaces_per_c:
        for s in spaces_per_c[c]:
            # get all adjacent spaces from the spaces in the compartment
            adj_spaces[c].extend(dict(sp_sp_adj)[s])
            # get the compartments the contained spaces are contained in
            con_compartments[c].append(space_in_c[s])

        #flatten the list of contained compartments
        con_compartments[c] = list(set(flatten(con_compartments[c])))

        # get the list of adjacent spaces by excluding spaces from list that are contained in current compartment (disjointment)
        adj_spaces[c] = [x for x in set(adj_spaces[c]) if x not in set(spaces_per_c[c])]

        # get the compartments the adjacent spaces are contained in
        adj_compartments[c] = list(set(flatten([space_in_c.get(s, []) for s in adj_spaces[c]])))

        # get the adjacent compartments by excluding compartments containing the spaces in the compartment from the list (disjointment)
        adj_compartments[c] = [x for x in adj_compartments.get(c, []) if x not in set(con_compartments[c])]

    # write compartment adjacency
    output = ""
    for c in adj_compartments.items():
        if c[1]:
            output += f"\ninst:Compartment_{str(c[0])}"
            first = True
            for adj_c in c[1]:
                if first:
                    first = False
                else:
                    output += " ;"
                output += "\n\tex:adjacentCompartment inst:Compartment_"+str(adj_c)
            output += " .\n"

    # Write space-space adjacencies
    for s in sp_sp_adj:
        output += f"\ninst:Space_{str(s[0])}"
        first = True
        for adj_s in s[1]:
            if first:
                first = False
            else:
                output += " ;"
            output += "\n\tbot:adjacentZone inst:Space_"+str(adj_s)

        output += " .\n"

    return(output)

### writeInterfaces

In [38]:
def writeInterfaces(model):
    output = ""
    for b in model.by_type("IfcRelSpaceBoundary"):
        output += "inst:interface_"+str(b.id()) + "\n"
        output += "\ta bot:Interface ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "

        sp = b.RelatingSpace
        el = b.RelatedBuildingElement
        if sp is not None:
            output += ";\n"
            output += "\tbot:interfaceOf inst:Space_"+ str(sp.id()) + " "
        if el is not None:
            el_type = el.is_a()
            output += ";\n"
            output += "\tbot:interfaceOf inst:" + el_type.replace("Ifc", "") + "_" + str(el.id()) + " "

        output += ". \n\n"
    return output

## Full conversion module

In [39]:
def IFC2TTL(input_ifc, output_ttl):
    # open file
    outputfile = open(output_ttl, "w").close() #clear file
    outputfile = open(output_ttl, "a") #add to file

    # define base URI
    now = datetime.now()
    current_time = now.strftime('%Y%m%d_%H%M%S')
    baseURI = "https://linkedbuildingdata.net/ifc/resources" + current_time + "/"
    #print("baseURI : " + baseURI)

    # creating a string to write
    output = "# baseURI: " + baseURI + "\n"
    output += f"@prefix inst: <{baseURI}> .\n"
    output += f"@prefix rdf: <{RDF}> .\n"
    output += f"@prefix rdfs: <{RDFS}> .\n"
    output += f"@prefix xsd: <{XSD}> .\n"
    #output += f"@prefix owl: <{OWL}> .\n"

    output += f"@prefix ifc: <{IFC}> .\n"
    output += f"@prefix bot: <{BOT}> .\n"
    output += f"@prefix beo: <{BEO}> .\n"
    output += f"@prefix ex: <{EX}> .\n"
    #output += f"@prefix : <{EMPTY}> .\n"

    #output += f"@prefix mep: <{MEP}> .\n"
    #output += f"@prefix geom: <{GEOM}> .\n"
    #output += f"@prefix mat: <{MAT}> .\n"

    output += f"@prefix props: <{PROPS}> .\n"
    output += "@prefix pset: <" + PSET + "> .\n"

    output += f"@prefix qudt:  <{QUDT}> .\n"
    output += f"@prefix unit: <{UNIT}> .\n"

    outputfile.write(output)

    output = ""

    # LOAD REQUIRED DATA
    print("Loading required data...")
    model = ios.open(input_ifc)
    beo_classes = loadBEO() # load BEO for type recognition
    model_units = findUnits(model) # extract units for standardization
    model_compartments = findCompartments(model) #extract names of compartments

    # WRITE DATA
    outputfile.write(RDF_header("SPATIAL ELEMENTS"))
    print("Writing spatial elements...")
    outputfile.write(writeSites(model, model_units))
    outputfile.write(writeBuildings(model, model_units, model_compartments))
    outputfile.write(writeStoreys(model, model_units, model_compartments))
    outputfile.write(writeCompartments(model, model_units, model_compartments))
    outputfile.write(writeSpaces(model, model_units, model_compartments))

    outputfile.write(RDF_header("BUILDING ELEMENTS"))
    print("Writing building elements...")
    outputfile.write(writeElements(model, model_units, beo_classes))

    outputfile.write(RDF_header("BUILDING MATERIALS"))
    print("Writing building materials...")
    outputfile.write(writeMaterials(model, model_units))

    outputfile.write(RDF_header("ADJACENCIES"))
    print("Writing adjacencies...")
    outputfile.write(writeAdjacencies(model, model_compartments))

    outputfile.write(RDF_header("INTERFACES"))
    print("Writing interfaces...")
    outputfile.write(writeInterfaces(model))
    outputfile.close()

    print("RDF export finished.")

---
# **Convert IFC file to TTL**

In [40]:
# convert correct use case model
IFC2TTL(input_correct, output_correct)

Loading required data...
Writing spatial elements...
ERROR CONVERTING IfcCountMeasure 2
Writing building elements...
Writing building materials...
ERROR CONVERTING IfcThermalConductivityMeasure 1.046
ERROR CONVERTING IfcThermalConductivityMeasure 0.167
ERROR CONVERTING IfcThermalConductivityMeasure 0.035
ERROR CONVERTING IfcThermalConductivityMeasure 0.025
ERROR CONVERTING IfcThermalConductivityMeasure 1.15
ERROR CONVERTING IfcThermalConductivityMeasure 1.046
ERROR CONVERTING IfcThermalConductivityMeasure 1.046
ERROR CONVERTING IfcThermalConductivityMeasure 0.54
ERROR CONVERTING IfcThermalConductivityMeasure 0.018999999999999996
ERROR CONVERTING IfcThermalConductivityMeasure 1.3
ERROR CONVERTING IfcThermalConductivityMeasure 0.51
ERROR CONVERTING IfcThermalConductivityMeasure 1.3
ERROR CONVERTING IfcThermalConductivityMeasure 0.65
ERROR CONVERTING IfcThermalConductivityMeasure 0.025
ERROR CONVERTING IfcThermalConductivityMeasure 1.1
ERROR CONVERTING IfcThermalConductivityMeasure 0.4999

In [41]:
# convert incorrect use case model
IFC2TTL(input_incorrect, output_incorrect)

Loading required data...
Writing spatial elements...
ERROR CONVERTING IfcCountMeasure 2
Writing building elements...
Writing building materials...
ERROR CONVERTING IfcThermalConductivityMeasure 1.046
ERROR CONVERTING IfcThermalConductivityMeasure 0.167
ERROR CONVERTING IfcThermalConductivityMeasure 0.035
ERROR CONVERTING IfcThermalConductivityMeasure 0.025
ERROR CONVERTING IfcThermalConductivityMeasure 1.15
ERROR CONVERTING IfcThermalConductivityMeasure 1.046
ERROR CONVERTING IfcThermalConductivityMeasure 1.046
ERROR CONVERTING IfcThermalConductivityMeasure 0.54
ERROR CONVERTING IfcThermalConductivityMeasure 0.018999999999999996
ERROR CONVERTING IfcThermalConductivityMeasure 1.3
ERROR CONVERTING IfcThermalConductivityMeasure 0.51
ERROR CONVERTING IfcThermalConductivityMeasure 1.3
ERROR CONVERTING IfcThermalConductivityMeasure 0.65
ERROR CONVERTING IfcThermalConductivityMeasure 0.025
ERROR CONVERTING IfcThermalConductivityMeasure 1.1
ERROR CONVERTING IfcThermalConductivityMeasure 0.4999

---
# **Full materialization**


Using full materialization based on the ontologies so that inference only has to be done once

In [43]:
# Load data graphs
data_graph_correct = Graph()
data_graph_correct.parse(output_correct, format="turtle")
data_graph_incorrect = Graph()
data_graph_incorrect.parse(output_incorrect, format="turtle")

<Graph identifier=N2b9e32abfb224733bd19e7871eb87c2d (<class 'rdflib.graph.Graph'>)>

In [45]:
# Prepare ontologies for materialization
ont_graph_isolated = Graph()
ont_graph_isolated.parse(ont_path_isolated, format="turtle")
ont_graph_alignment = Graph()
ont_graph_alignment.parse(ont_path_alignment, format="turtle")

<Graph identifier=Nec028a4878314ef9979b2337a563c9d0 (<class 'rdflib.graph.Graph'>)>

In [46]:
# Materialize inferences in-place
data_graph_correct_materialized = data_graph_correct
conforms, report_graph, report_text = pyshacl.validate(
    data_graph_correct_materialized,
    shacl_graph=None,        # no shapes needed for inference
    ont_graph=ont_graph_isolated + ont_graph_alignment,
    inference='owlrl',       # Apply RDFS or OWL-RL reasoning (or both)
    inplace=True,            # mutate data_graph to include inferred triples
    serialize_report_graph=False
)

# Materialize inferences in-place
data_graph_incorrect_materialized = data_graph_incorrect
conforms, report_graph, report_text = pyshacl.validate(
    data_graph_incorrect_materialized,
    shacl_graph=None,        # no shapes needed for inference
    ont_graph=ont_graph_isolated + ont_graph_alignment,
    inference='owlrl',       # Apply RDFS or OWL-RL reasoning (or both)
    inplace=True,            # mutate data_graph to include inferred triples
    serialize_report_graph=False
)

# Save the materialized graphs
data_graph_correct_materialized.serialize(destination=output_correct.replace('.ttl', '_materialized.ttl'), format="turtle")
data_graph_incorrect_materialized.serialize(destination=output_incorrect.replace('.ttl', '_materialized.ttl'), format="turtle")

# Preview the materialized graphs
print(data_graph_correct_materialized.serialize(format="turtle")[800:1400])
print(data_graph_incorrect_materialized.serialize(format="turtle")[800:1400])

t .

rdf:HTML a rdfs:Datatype ;
    owl:sameAs rdf:HTML .

rdf:PlainLiteral a rdfs:Datatype ;
    owl:sameAs rdf:PlainLiteral .

rdf:XMLLiteral a rdfs:Datatype ;
    owl:sameAs rdf:XMLLiteral .

rdf:langString a rdfs:Datatype ;
    owl:sameAs rdf:langString .

rdf:type owl:sameAs rdf:type .

rdfs:Literal a rdfs:Datatype ;
    owl:sameAs rdfs:Literal .

rdfs:comment a owl:AnnotationProperty ;
    owl:sameAs rdfs:comment .

rdfs:domain owl:sameAs rdfs:domain .

rdfs:isDefinedBy a owl:AnnotationProperty ;
    owl:sameAs rdfs:isDefinedBy .

rdfs:label a owl:AnnotationProperty ;
    owl:sameAs rdfs
t .

rdf:HTML a rdfs:Datatype ;
    owl:sameAs rdf:HTML .

rdf:PlainLiteral a rdfs:Datatype ;
    owl:sameAs rdf:PlainLiteral .

rdf:XMLLiteral a rdfs:Datatype ;
    owl:sameAs rdf:XMLLiteral .

rdf:langString a rdfs:Datatype ;
    owl:sameAs rdf:langString .

rdf:type owl:sameAs rdf:type .

rdfs:Literal a rdfs:Datatype ;
    owl:sameAs rdfs:Literal .

rdfs:comment a owl:AnnotationProperty ;
    